In [34]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Exploring-Mental-Health-Data.git

# Change directory to the cloned repository
%cd Exploring-Mental-Health-Data

Cloning into 'Exploring-Mental-Health-Data'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 42 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 5.49 MiB | 3.78 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data


In [35]:
import pandas as pd
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
!pip install scikit-optimize
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
import pandas as pd
from sklearn.metrics import classification_report

In [36]:
train_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/train.csv')
test_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/test.csv')

In [37]:
train_set.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


# Data Inspection

In [38]:
#train_print = train_set.copy()
#train_print = train_print.drop(columns=['Depression'])

#for column in train_print.columns:
    #train_nan_ratio = train_print[column].isna().sum() / len(train_print)  # Proportion of NaN in train set
    #test_nan_ratio = test_set[column].isna().sum() / len(test_set)  # Proportion of NaN in test set
    #print(f"{column}  Train Set: {train_nan_ratio:.2%} Test Set: {test_nan_ratio:.2%}")

In [39]:
# Get the value counts for train and test sets
#train_counts = train_set['Profession'].value_counts()
#test_counts = test_set['Profession'].value_counts()

# Get a combined set of unique values from both train and test sets
#all_professions = set(train_counts.index).union(set(test_counts.index))

# Print the counts for each profession
#for profession in all_professions:
 #   train_count = train_counts.get(profession, 0)  # Get count from train, default to 0
  #  test_count = test_counts.get(profession, 0)   # Get count from test, default to 0
   # print(f"{profession}: Train_set={train_count} Test_set={test_count}")

# Feature Engineering:


In [40]:
train_id = train_set['id']

target_variable = train_set['Depression']
train_set.drop(columns=['Name','Depression'], inplace=True)
test_id = test_set['id']

test_set.drop(columns=['Name'], inplace=True)

train_set['Gender'] = train_set['Gender'].map({'Male' : 1, 'Female' : 0}) # Always trasnforming 2 option object types into binary
test_set['Gender'] = test_set['Gender'].map({'Male' : 1, 'Female' : 0})

train_set['Age'] = train_set['Age'] / 100 # Normalized by the Max value
train_set['Age'] = train_set['Age'].fillna(train_set['Age'].mean())
test_set['Age'] = test_set['Age'] / 100
test_set['Age'] = test_set['Age'].fillna(test_set['Age'].mean())

train_set['Working Professional or Student'] = train_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0}) # Always trasnforming 2 option object types into binary
test_set['Working Professional or Student'] = test_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0})

train_set['CGPA'] = train_set['CGPA'] /10
train_set['CGPA'] = train_set['CGPA'].fillna(0)
test_set['CGPA'] = test_set['CGPA'] /10
test_set['CGPA'] = test_set['CGPA'].fillna(0)

train_set['Have you ever had suicidal thoughts ?'] = train_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0}) # Always trasnforming 2 option object types into binary
test_set['Have you ever had suicidal thoughts ?'] = test_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0})

train_set['Work/Study Hours'] = train_set['Work/Study Hours'] / 24 # Normalized by the Max value
test_set['Work/Study Hours'] = test_set['Work/Study Hours'] / 24

train_set['Financial Stress'] = train_set['Financial Stress'] / 5 # Normalized by the Max value
train_set['Financial Stress'] = train_set['Financial Stress'].fillna(0)
test_set['Financial Stress'] = test_set['Financial Stress'] / 5
test_set['Financial Stress'] = test_set['Financial Stress'].fillna(0)

train_set['Family History of Mental Illness'] = train_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0}) # Always trasnforming 2 option object types into binary
test_set['Family History of Mental Illness'] = test_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0})

In [41]:
# Create the new feature 'Academic/Work Pressure'
train_set['Academic/Work Pressure'] = train_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
train_set['Academic/Work Pressure'] = train_set['Academic/Work Pressure'].fillna(train_set['Academic/Work Pressure'].mean())

train_set['Academic Pressure'] = train_set['Academic Pressure'] / 5
train_set['Academic Pressure'] = train_set['Academic Pressure'].fillna(0)

train_set['Work Pressure'] = train_set['Work Pressure'] / 5
train_set['Work Pressure'] = train_set['Work Pressure'].fillna(0)

In [42]:
# Create the new feature 'Academic/Work Pressure'
test_set['Academic/Work Pressure'] = test_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
test_set['Academic/Work Pressure'] = test_set['Academic/Work Pressure'].fillna(test_set['Academic/Work Pressure'].mean())

test_set['Academic Pressure'] = test_set['Academic Pressure'] / 5
test_set['Academic Pressure'] = test_set['Academic Pressure'].fillna(0)

test_set['Work Pressure'] = test_set['Work Pressure'] / 5
test_set['Work Pressure'] = test_set['Work Pressure'].fillna(0)

In [43]:
# Create the new feature Study/Job Satisfaction
train_set['Study/Job Satisfaction'] = train_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in Study/Job Satisfaction with its mean
train_set['Study/Job Satisfaction'] = train_set['Study/Job Satisfaction'].fillna(train_set['Study/Job Satisfaction'].mean())

train_set['Study Satisfaction'] = train_set['Study Satisfaction'] / 5
train_set['Study Satisfaction'] = train_set['Study Satisfaction'].fillna(0)

train_set['Job Satisfaction'] = train_set['Job Satisfaction'] / 5
train_set['Job Satisfaction'] = train_set['Job Satisfaction'].fillna(0)

In [44]:
# Create the new feature Study/Job Satisfaction
test_set['Study/Job Satisfaction'] = test_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1)

# Fill any remaining NaN in Study/Job Satisfaction with its mean
test_set['Study/Job Satisfaction'] = test_set['Study/Job Satisfaction'].fillna(test_set['Study/Job Satisfaction'].mean())

test_set['Study Satisfaction'] = test_set['Study Satisfaction'] / 5
test_set['Study Satisfaction'] = test_set['Study Satisfaction'].fillna(0)

test_set['Job Satisfaction'] = test_set['Job Satisfaction'] / 5
test_set['Job Satisfaction'] = test_set['Job Satisfaction'].fillna(0)

In [45]:
# Combine train and test value counts
train_counts = train_set['City'].value_counts()
test_counts = test_set['City'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['City'] = train_set['City'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['City'] = test_set['City'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [46]:
# Combine train and test value counts
train_counts = train_set['Profession'].value_counts()
test_counts = test_set['Profession'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['Profession'] = train_set['Profession'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['Profession'] = test_set['Profession'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [47]:
# Combine train and test value counts
train_counts = train_set['Degree'].value_counts()
test_counts = test_set['Degree'].value_counts()

# Create a combined count of professions
combined_counts = train_counts.add(test_counts, fill_value=0)

# Identify professions appearing less than 5 times in either set
low_frequency_values = combined_counts[combined_counts < 10].index.tolist()

# Map low-frequency values to 'None' in both train and test sets
train_set['Degree'] = train_set['Degree'].apply(lambda x: 'Other' if x in low_frequency_values else x)
test_set['Degree'] = test_set['Degree'].apply(lambda x: 'Other' if x in low_frequency_values else x)

In [48]:
train_set['City'] = train_set['City'].fillna('X')
freq = train_set['City'].value_counts()
train_set['City_freq'] = train_set['City'].map(freq) # Frequency encoding different cities

test_set['City'] = test_set['City'].fillna('X')
test_set['City_freq'] = test_set['City'].map(freq) # Frequency encoding different cities


train_set['Degree'] = train_set['Degree'].fillna('X')
freq = train_set['Degree'].value_counts()
train_set['Degree_freq'] = train_set['Degree'].map(freq) # Frequency encoding

test_set['Degree'] = test_set['Degree'].fillna('X')
test_set['Degree_freq'] = test_set['Degree'].map(freq) # Frequency encoding


train_set['Profession'] = train_set['Profession'].fillna('X')
freq = train_set['Profession'].value_counts()
train_set['Profession_freq'] = train_set['Profession'].map(freq) # Frequency encoding

test_set['Profession'] = test_set['Profession'].fillna('X')
test_set['Profession_freq'] = test_set['Profession'].map(freq) # Frequency encoding

In [49]:
# Save the original number of rows for splitting later
train_size = len(train_set)

# Concatenate train and test sets
concat_set = pd.concat([train_set, test_set], axis=0)

# Apply one-hot encoding to the concatenated set
concat_set = pd.get_dummies(concat_set, columns=['Degree', 'Profession', 'City'])

# Split the concatenated set back into train and test sets
train_set = concat_set.iloc[:train_size, :].copy()  # Get the original train rows
test_set = concat_set.iloc[train_size:, :].copy()   # Get the original test rows

print("Train set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

Train set shape: (140700, 119)
Test set shape: (93800, 119)


In [50]:
 #Define a function to encode Sleep Duration
def encode_sleep_duration(value):
    # Check for NaN or None
    if pd.isna(value):
        return -1  # Placeholder for missing or invalid values

    # Ensure the value is a string
    value = str(value).strip()

    # Encoding logic
    if 'Less than 5 hours' in value or value in ['3-4 hours', '4-5 hours', '2-3 hours']:
        return 1  # < 5 hours
    elif '5-6 hours' in value:
        return 2  # 5-6 hours
    elif '6-7 hours' in value or '6-8 hours' in value:
        return 3  # 6-7 hours
    elif '7-8 hours' in value:
        return 4  # 7-8 hours
    elif 'More than 8 hours' in value or '8-9 hours' in value or '9-11 hours' in value:
        return 5  # > 8 hours
    else:
        return -1  # Placeholder for other invalid or unclear values

sleep_mapping = {
    'More than 8 hours': '>8',
    'Less than 5 hours': '<5',
    'Moderate': '5-6 hours',
    '9-6 hours': '6-9 hours',

    '10-6 hours': '6-10 hours',
    'than 5 hours': '4-6 hours',
    'Unhealthy': '4-6 hours',
    '9-5 hours': '5-9 hours',
    '9-5': '5-9 hours',
    '8-89 hours': '8-9 hours',

    # Ambiguous or irrelevant entries
    'Sleep_Duration': None, '40-45 hours': None, '55-66 hours': None, 'Indore': None,
    '45': None, '35-36 hours': None, 'No': None, 'Indore': None, '49 hours': None,
    'Work_Study_Hours': None, '45-48 hours': None, 'Pune': None, 'Soham': None,
    '0': None, 'Meerut': None, '60-65 hours': None, 'Vivan': None,
    'Have_you_ever_had_suicidal_thoughts': None, '20-21 hours': None, '50-75 hours': None
}

# Define range extraction function
def extract_range(value):
    try:
        if '-' in value:
            parts = value.split('-')
            return int(parts[0]), int(parts[1].split()[0])
        elif '<' in value:
            return 3, 5
        elif '>' in value:
            lower_bound = int(value.split('>')[1].split()[0])
            return lower_bound, 10
        elif value.isdigit():
            num = int(value)
            return num, num
        else:
            return None, None
    except:
        return None, None

In [51]:
train_set['Sleep Duration'] = train_set['Sleep Duration'].map(sleep_mapping)
test_set['Sleep Duration'] = test_set['Sleep Duration'].map(sleep_mapping)

train_set['Sleep_Duration_Encoded'] = train_set['Sleep Duration'].apply(encode_sleep_duration)
test_set['Sleep_Duration_Encoded'] = test_set['Sleep Duration'].apply(encode_sleep_duration)

# Apply the function to create 'From' and 'To' columns
train_set[['From', 'To']] = train_set['Sleep Duration'].apply(lambda x: pd.Series(extract_range(x)))
test_set[['From', 'To']] = test_set['Sleep Duration'].apply(lambda x: pd.Series(extract_range(x)))

# Handle missing values
train_set['From'] = train_set['From'].fillna(train_set['From'].mean())
train_set['To'] = train_set['To'].fillna(train_set['To'].mean())

test_set['From'] = test_set['From'].fillna(test_set['From'].mean())
test_set['To'] = test_set['To'].fillna(test_set['To'].mean())

# Drop the original column if not needed
train_set.drop(columns=['Sleep Duration'], inplace=True)
test_set.drop(columns=['Sleep Duration'], inplace=True)

In [52]:
print("Train set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

Train set shape: (140700, 121)
Test set shape: (93800, 121)


In [53]:
# Define mapping for valid dietary habits
dietary_mapping = {
    'Moderate': 'Moderate',
    'Unhealthy': 'Unhealthy',
    'Healthy': 'Healthy',
    'More Healthy': 'Healthy',
    'Less Healthy': 'Unhealthy',
    'Less than Healthy': 'Unhealthy',
    'No Healthy': 'Unhealthy',
    '3': 'Moderate',
    '1.0': 'Unhealthy',
    '2': 'Moderate',
    '5 Healthy': 'Healthy',
    '5 Unhealthy': 'Unhealthy',

    # Ambiguous or irrelevant entries
    'Yes': None, 'No': None, 'No Healthy': None, 'Class 12': None, 'Indoor': None,
    'Male': None, 'Vegas': None, 'M.Tech': None, 'Electrician': None,
    'Hormonal': None, 'Mihir': None, 'Gender': None, 'BSc': None,
    'Pratham': None, 'Prachi': None, 'Resistant': None, 'Mealy': None,

    'nan': None, 'Academic': None, 'Educational': None, 'Soham': None,
    'Naina': None, 'Kolkata': None, 'Raghav': None, 'Vivaan': None,  'MCA': None,
}

# Apply the mapping to the Dietary Habits column
train_set['Dietary Habits'] = train_set['Dietary Habits'].map(dietary_mapping)
train_set['Dietary Habits'] = train_set['Dietary Habits'].fillna('X')

test_set['Dietary Habits'] = test_set['Dietary Habits'].map(dietary_mapping)
test_set['Dietary Habits'] = test_set['Dietary Habits'].fillna('X')

freq = train_set['Dietary Habits'].value_counts()
train_set['Dietary Habits_freq'] = train_set['Dietary Habits'].map(freq) # frequency encoding
test_set['Dietary Habits_freq'] = test_set['Dietary Habits'].map(freq) # frequency encoding

# Save the original number of rows for splitting later
train_size = len(train_set)

# Concatenate train and test sets
concat_set = pd.concat([train_set, test_set], axis=0)

# Apply one-hot encoding to the concatenated set
concat_set = pd.get_dummies(concat_set, columns=['Dietary Habits'])

# Split the concatenated set back into train and test sets
train_set = concat_set.iloc[:train_size, :].copy()  # Get the original train rows
test_set = concat_set.iloc[train_size:, :].copy()   # Get the original test rows

In [54]:
print("Train set shape:", train_set.shape)
print("Test set shape:", test_set.shape)

Train set shape: (140700, 125)
Test set shape: (93800, 125)


In [55]:
train_set.to_csv('/content/Exploring-Mental-Health-Data/data/PP_train.csv', index=False)
test_set.to_csv('/content/Exploring-Mental-Health-Data/data/PP_test.csv', index=False)

In [22]:
# Check for columns with object data type in train_set and test_set
train_object_columns = train_set.select_dtypes(include=['object']).columns
test_object_columns = test_set.select_dtypes(include=['object']).columns

print("Object columns in train_set:", train_object_columns.tolist())
print("Object columns in test_set:", test_object_columns.tolist())

# Check for columns with missing values in train_set and test_set
train_missing = train_set.isna().sum()
test_missing = test_set.isna().sum()

print("\nColumns with missing values in train_set:")
print(train_missing[train_missing > 0])

print("\nColumns with missing values in test_set:")
print(test_missing[test_missing > 0])


Object columns in train_set: []
Object columns in test_set: []

Columns with missing values in train_set:
Series([], dtype: int64)

Columns with missing values in test_set:
Series([], dtype: int64)


In [23]:
X = train_set.copy()
y = target_variable

In [24]:
model = XGBClassifier()
model.fit(X, y)

feature_importances = model.feature_importances_
important_features = X.columns[feature_importances > 0.005]
X_reduced = X[important_features]
X_reduced.columns

Index(['Age', 'Academic Pressure', 'Work Pressure', 'Study Satisfaction',
       'Job Satisfaction', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress', 'Academic/Work Pressure',
       'Study/Job Satisfaction', 'Profession_freq', 'Profession_Chemist',
       'Profession_Content Writer', 'Profession_Entrepreneur',
       'Profession_Judge', 'Profession_Pharmacist', 'City_Ahmedabad', 'From',
       'To', 'Dietary Habits_freq', 'Dietary Habits_Healthy',
       'Dietary Habits_Unhealthy'],
      dtype='object')

In [25]:
pca = PCA(n_components=5)
X_reduced = pca.fit_transform(X)

X_reduced_df = pd.DataFrame(X_reduced, columns=[f'PC{i+1}' for i in range(X_reduced.shape[1])])

# Get explained variance ratio
explained_variance = pca.explained_variance_ratio_

# Display the cumulative variance
cumulative_variance = explained_variance.cumsum()

# Print explained variance and cumulative variance
for i, var in enumerate(explained_variance):
    print(f"Principal Component {i+1}: {var:.4f} (Cumulative: {cumulative_variance[i]:.4f})")

Principal Component 1: 0.8736 (Cumulative: 0.8736)
Principal Component 2: 0.1180 (Cumulative: 0.9916)
Principal Component 3: 0.0055 (Cumulative: 0.9971)
Principal Component 4: 0.0025 (Cumulative: 0.9996)
Principal Component 5: 0.0004 (Cumulative: 1.0000)


In [26]:
model = Lasso(alpha=0.01)
model.fit(X, y)
important_features = X.columns[model.coef_ != 0]
X_reduced = X[important_features]
X_reduced.columns

Index(['id', 'Academic Pressure', 'Have you ever had suicidal thoughts ?',
       'Financial Stress', 'Academic/Work Pressure', 'Study/Job Satisfaction',
       'City_freq', 'Degree_freq', 'Profession_freq', 'Profession_Teacher',
       'From', 'To', 'Dietary Habits_freq', 'Dietary Habits_Unhealthy'],
      dtype='object')

In [27]:
X = train_set[['City_freq', 'Profession_freq',
       'Profession_Teacher', 'Degree_freq', 'Dietary Habits_freq',
       'Dietary Habits_Unhealthy', 'Age', 'Academic Pressure', 'Work Pressure', 'Study Satisfaction',
       'Job Satisfaction', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Profession_Architect', 'Profession_Chemist',
       'Profession_Content Writer', 'Profession_Entrepreneur',
       'Profession_Judge', 'Profession_Pharmacist', 'Academic/Work Pressure',
       'Study/Job Satisfaction', 'From', 'To',
       'Dietary Habits_Healthy']] + X_reduced_df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']]

In [28]:
X = train_set.copy()
y = target_variable

# Initial split for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Print the count of True and False in both train and validation sets
print("Training set class distribution:\n", y_train.value_counts())
print("Validation set class distribution:\n", y_val.value_counts())

Training set class distribution:
 Depression
0    103620
1     23010
Name: count, dtype: int64
Validation set class distribution:
 Depression
0    11513
1     2557
Name: count, dtype: int64


In [29]:
from sklearn.metrics import classification_report
import xgboost as xgb

# List of model parameters
model_parameters = [
    {'colsample_bytree': 1.0, 'gamma': 0.0, 'learning_rate': 0.14053992185033634, 'max_depth': 2, 'n_estimators': 500, 'reg_alpha': 3.54988240208208, 'reg_lambda': 0.921308795529119, 'subsample': 0.5},
    {'colsample_bytree': 0.9828642489404322, 'gamma': 3.8356539257184212, 'learning_rate': 0.24821760051945593, 'max_depth': 2, 'n_estimators': 496, 'reg_alpha': 1.9768655910871376, 'reg_lambda': 10.0, 'subsample': 0.6876446417339648},
    {'colsample_bytree': 0.5482548720792537, 'gamma': 0.7585910862891333, 'learning_rate': 0.24362567776750851, 'max_depth': 2, 'n_estimators': 399, 'reg_alpha': 9.183318680337436, 'reg_lambda': 8.354951719581466, 'subsample': 0.9288717269361884},
    {'colsample_bytree': 0.826258443653563, 'gamma': 0.2898189814539954, 'learning_rate': 0.1560857716723225, 'max_depth': 3, 'n_estimators': 252, 'reg_alpha': 9.909946019462808, 'reg_lambda': 8.794138788777472, 'subsample': 0.6086089653039243},
    {'colsample_bytree': 0.5, 'gamma': 1.0613803072595367, 'learning_rate': 0.4309050398635545, 'max_depth': 2, 'n_estimators': 491, 'reg_alpha': 9.395079774043808, 'reg_lambda': 7.331815596626036, 'subsample': 0.5},
    {'colsample_bytree': 1.0, 'gamma': 0.8055969826831805, 'learning_rate': 0.07850322110212825, 'max_depth': 2, 'n_estimators': 500, 'reg_alpha': 10.0, 'reg_lambda': 7.307261693189616, 'subsample': 0.5},
    {'colsample_bytree': 0.5071052160640582, 'gamma': 6.532630659116698, 'learning_rate': 0.05917416968889502, 'max_depth': 41, 'n_estimators': 500, 'reg_alpha': 5.525276688982408, 'reg_lambda': 10.0, 'subsample': 0.8337729014389854},
    {'colsample_bytree': 0.8682156936651646, 'gamma': 2.3661463968930123, 'learning_rate': 0.5, 'max_depth': 2, 'n_estimators': 500, 'reg_alpha': 6.878059550769972, 'reg_lambda': 10.0, 'subsample': 0.5},
    {'colsample_bytree': 0.5038829468838858, 'gamma': 1.4575211520278655, 'learning_rate': 0.06985053383947529, 'max_depth': 6, 'n_estimators': 414, 'reg_alpha': 9.499001815481689, 'reg_lambda': 0.9976351956145648, 'subsample': 0.7122725319242551},
    {'colsample_bytree': 0.9505559305966407, 'gamma': 0.0, 'learning_rate': 0.31034434048228005, 'max_depth': 2, 'n_estimators': 407, 'reg_alpha': 8.566301647743709, 'reg_lambda': 10.0, 'subsample': 0.6381124861311176},
    {'colsample_bytree': 0.5271397711907289, 'gamma': 1.4882692983778134, 'learning_rate': 0.022613833933071845, 'max_depth': 70, 'n_estimators': 500, 'reg_alpha': 7.350122507234383, 'reg_lambda': 2.3467446363910205, 'subsample': 1.0},
    {'colsample_bytree': 0.968269317216317, 'gamma': 0.7956592762847293, 'learning_rate': 0.22241747714939272, 'max_depth': 6, 'n_estimators': 268, 'reg_alpha': 9.61123872314133, 'reg_lambda': 1.0199763042557133, 'subsample': 0.9101921473710248},
    {'colsample_bytree': 0.5010496053339198, 'gamma': 0.7546323022179792, 'learning_rate': 0.01860526126199002, 'max_depth': 12, 'n_estimators': 499, 'reg_alpha': 8.720527262017999, 'reg_lambda': 1.3463384912771639, 'subsample': 0.9265861439852696},
    {'colsample_bytree': 0.5510220279765351, 'gamma': 7.984488731903994, 'learning_rate': 0.09183744558408916, 'max_depth': 60, 'n_estimators': 380, 'reg_alpha': 3.1539117758914, 'reg_lambda': 1.7690590466474512, 'subsample': 0.555548765738129},
    {'colsample_bytree': 0.543245954963853, 'gamma': 2.7342452805864643, 'learning_rate': 0.04318592293247114, 'max_depth': 46, 'n_estimators': 362, 'reg_alpha': 4.0009466491413335, 'reg_lambda': 1.5685416376985208, 'subsample': 0.8836643772841459},
]

# Train and evaluate each model
for i, params in enumerate(model_parameters, start=1):
    print(f"Training Model {i}...")
    model = xgb.XGBClassifier(**params, eval_metric='error')
    model.fit(X_train, y_train)

    # Predictions on train and validation sets
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(test_set)

    # Print classification reports
    print(f"Classification Report - Train (Model {i}):")
    print(classification_report(y_train, y_train_pred))

    print(f"Classification Report - Validation (Model {i}):")
    print(classification_report(y_val, y_val_pred))


Training Model 1...
Classification Report - Train (Model 1):
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    103620
           1       0.85      0.82      0.84     23010

    accuracy                           0.94    126630
   macro avg       0.91      0.89      0.90    126630
weighted avg       0.94      0.94      0.94    126630

Classification Report - Validation (Model 1):
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     11513
           1       0.84      0.82      0.83      2557

    accuracy                           0.94     14070
   macro avg       0.90      0.89      0.90     14070
weighted avg       0.94      0.94      0.94     14070

Training Model 2...
Classification Report - Train (Model 2):
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    103620
           1       0.85      0.82      0.83     23010

    accuracy  

In [105]:
# Define parameter search space
search_spaces = {
    'learning_rate': Real(0.001, 0.5, 'uniform'),
    'max_depth': Integer(2, 70),
    'n_estimators': Integer(100, 500),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.5, 1.0, 'uniform'),
    'gamma': Real(0, 10, 'uniform'),
    'reg_alpha': Real(0, 10, 'uniform'),
    'reg_lambda': Real(0, 10, 'uniform')
}

# Initialize the XGBClassifier model with GPU support
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Use for binary classification
    tree_method='hist',  # Use GPU for training
    verbosity=2
)

kf = KFold(n_splits=5)  # 5-fold cross-validation

# Use 'accuracy' as the scoring metric
optimizer = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=256,
    cv=kf,
    scoring='accuracy',  # Change scoring metric to accuracy
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)

# Fit the optimizer using X_train and y_train
optimizer.fit(X_train, y_train)

# Get cross-validation results
cv_results = optimizer.cv_results_

# Create a DataFrame to sort and filter the best models
results_df = pd.DataFrame(cv_results)
top_15_results = results_df.nlargest(15, 'mean_test_score')  # Get top 5 models by mean test score

# Train each of the top 5 models on the full training data and save them
top_15_models = []
for i, row in top_15_results.iterrows():
    params = row['params']
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        tree_method='hist',
        **params
    )
    model.fit(X_train, y_train)
    top_15_models.append(model)

# Print parameters and scores for the top 5 models
for idx, model in enumerate(top_15_models):
    print(f"Model {idx + 1} parameters: {top_15_results.iloc[idx]['params']}")
    print(f"Model {idx + 1} CV score: {top_15_results.iloc[idx]['mean_test_score']}")

print("Top 5 XGBoost models based on accuracy have been trained and saved.")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Model 1 parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 0.0), ('learning_rate', 0.14053992185033634), ('max_depth', 2), ('n_estimators', 500), ('reg_alpha', 3.54988240208208), ('reg_lambda', 0.921308795529119), ('subsample', 0.5)])
Model 1 CV score: 0.9397378188422965
Model 2 parameters: OrderedDict([('colsample_bytree', 0.9828642489404322), ('gamma', 3.8356539257184212), ('learning_rate', 0.24821760051945593), ('max_depth', 2), ('n_estimators', 496),

In [32]:
from sklearn.metrics import classification_report
import xgboost as xgb

params1 = {
    'colsample_bytree': 1.0,
    'gamma': 0.0,
    'learning_rate': 0.14053992185033634,
    'max_depth': 2,
    'n_estimators': 500,
    'reg_alpha': 3.54988240208208,
    'reg_lambda': 0.921308795529119,
    'subsample': 0.5
}
# Define the model with the provided parameters
xgb_model1 = xgb.XGBClassifier(**params1, eval_metric='error')

# Train the model
xgb_model1.fit(X_train, y_train)

# Predictions on train set
y_pred_prob_XGB1_train = xgb_model1.predict(X_train)
print("Classification Report - Train:")
print(classification_report(y_train, y_pred_prob_XGB1_train))

# Predictions on validation set
y_pred_prob_XGB1_val = xgb_model1.predict(X_val)
print("Classification Report - Validation:")
print(classification_report(y_val, y_pred_prob_XGB1_val))

y_pred_prob_XGB1_test = xgb_model1.predict(test_set)

Classification Report - Train:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96    103620
           1       0.85      0.82      0.84     23010

    accuracy                           0.94    126630
   macro avg       0.91      0.89      0.90    126630
weighted avg       0.94      0.94      0.94    126630

Classification Report - Validation:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     11513
           1       0.84      0.82      0.83      2557

    accuracy                           0.94     14070
   macro avg       0.90      0.89      0.90     14070
weighted avg       0.94      0.94      0.94     14070



In [33]:
from google.colab import files
# Prepare the output DataFrame
output = pd.DataFrame({'id': test_id.values, 'loan_status': y_pred_prob_XGB1_test})

# Remove any duplicate rows by 'PassengerId'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('predictions.csv', index=False)
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>